# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [ ]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4

In [3]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [ ]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer

In [4]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\נאוה ששון\Desktop\מדעי המחשב\שנה ב\סמסטר ב\למידת מכונה\מטלות\מטלה ניתוח טקסט\assignment5-text_analysis


In [ ]:
pip install --upgrade threadpoolctl


In [ ]:
pip install --upgrade scikit-learn


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [5]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [6]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [7]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [8]:
hebrew_text= df_train.loc[[0], 'story'].to_string()
hebrew_text

'0    כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...'

In [9]:
def my_tokenizer_function(hebrew_text):
    list_of_words= list()
    tokens = ht.tokenize(hebrew_text)  # tokenize returns a generator!
    for grp, token, token_num, (start_index, end_index) in tokens:
        if grp == 'HEBREW':
            list_of_words.append(token)
    return list_of_words

In [10]:
df_train_new= df_train.copy()
#replace the gender collumn to categorial
replace_map_gender= {'m':0, 'f': 1}
# df_train_new['gender']= df_train_new['gender'].astype('category')
df_train_new.replace(replace_map_gender, inplace= True)
df_train_new.head(5)

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",0
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,0
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,1
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",0
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,1


In [11]:
# split the df to X_train and y_train for vectorization:
X_train = df_train_new['story']
y_train = df_train_new['gender']

In [12]:
X_train.head(5)
y_train.head(5)

0    כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...
1    לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...
2    מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...
3    כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...
4    ‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...
Name: story, dtype: object

0    0
1    0
2    1
3    0
4    1
Name: gender, dtype: int64

# Vectorization

In [13]:
# from text_analysis_exercise
#kmeans
kmeans_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer_function)),
    ('norm', preprocessing.Normalizer(norm='l2')),
    ('kmeans', KMeans(n_clusters=5))
])

#decision tree
decision_tree_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer_function)),
    ('norm', preprocessing.Normalizer(norm='l2')),
    ('decision_tree', DecisionTreeClassifier())
])

# Naive bayes
NB_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer_function)),
    ('norm', preprocessing.Normalizer(norm='l2')),  #בהרצאה אמרו שעדיף l2 מאשר l1
    ('clf', MultinomialNB()),
])

#Lineaer SVC
svc_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer_function)),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('classifier', LinearSVC(penalty='l2'))
])

#SVM
SGDClassifier_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer_function)),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('clf', SGDClassifier(max_iter=1000, random_state=42))
])

# define param grids:
kmeans_param_grid = {
    'tfidf__max_features': [5000, 10000],
    'kmeans__n_clusters': [5, 10, 15],
    'kmeans__init': ['k-means++', 'random'],
    'kmeans__n_init': [5, 10],
    'kmeans__max_iter': [100, 300]
}

decision_tree_param_grid = {
    'tfidf__max_features': [5000, 10000],
    'decision_tree__criterion': ['gini', 'entropy'],
    'decision_tree__max_depth': [None, 10, 20],
    'decision_tree__min_samples_split': [2, 5],
    'decision_tree__min_samples_leaf': [1, 3]
}

NB_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1]
}

SGDClassifier_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
    'clf__penalty': ['l1', 'l2', 'elasticnet'],
    'clf__max_iter': [1000, 2000, 3000]
}

svc_param_grid = {
    'tfidf__max_df': [0.5, 0.75, 1.0],
    'classifier__C': [0.1, 1, 10],
    'classifier__max_iter': [1000, 2000, 3000]
}

# Define list of pipelines:
pipelines = [
    ('Kmeans', kmeans_pipeline, kmeans_param_grid),
    ('Decision Trees', decision_tree_pipeline, decision_tree_param_grid),
    ('SGDClassifier', SGDClassifier_pipeline, SGDClassifier_param_grid),
    ('Naive Bayes', NB_pipeline, NB_param_grid),
    ('Linear SVC', svc_pipeline, svc_param_grid)
]

best_model = None
best_f1_score = 0

In [14]:
for name, pipeline, param_grid in pipelines:
    print(f"evaluating {name}...")
    grid_search = GridSearchCV(pipeline, param_grid, cv=10,scoring='f1_macro', error_score='raise')
    grid_search.fit(X_train, y_train)
    print(f"Best hyperparameters: {grid_search.best_params_}")
    print("CV score=%0.3f" % grid_search.best_score_)
    if grid_search.best_score_ > best_f1_score:
        best_f1_score = grid_search.best_score_
        best_model = grid_search.best_estimator_

# Print the best model
print(f"The best model is: {best_model} with f1 score of: {best_f1_score}.")

evaluating Kmeans...


GridSearchCV(cv=10, error_score='raise',
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function my_tokenizer_function at 0x00000208A53F6310>)),
                                       ('norm', Normalizer()),
                                       ('kmeans', KMeans(n_clusters=5))]),
             param_grid={'kmeans__init': ['k-means++', 'random'],
                         'kmeans__max_iter': [100, 300],
                         'kmeans__n_clusters': [5, 10, 15],
                         'kmeans__n_init': [5, 10],
                         'tfidf__max_features': [5000, 10000]},
             scoring='f1_macro')

Best hyperparameters: {'kmeans__init': 'random', 'kmeans__max_iter': 300, 'kmeans__n_clusters': 5, 'kmeans__n_init': 10, 'tfidf__max_features': 5000}
CV score=0.117
evaluating Decision Trees...


GridSearchCV(cv=10, error_score='raise',
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function my_tokenizer_function at 0x00000208A53F6310>)),
                                       ('norm', Normalizer()),
                                       ('decision_tree',
                                        DecisionTreeClassifier())]),
             param_grid={'decision_tree__criterion': ['gini', 'entropy'],
                         'decision_tree__max_depth': [None, 10, 20],
                         'decision_tree__min_samples_leaf': [1, 3],
                         'decision_tree__min_samples_split': [2, 5],
                         'tfidf__max_features': [5000, 10000]},
             scoring='f1_macro')

Best hyperparameters: {'decision_tree__criterion': 'gini', 'decision_tree__max_depth': None, 'decision_tree__min_samples_leaf': 3, 'decision_tree__min_samples_split': 2, 'tfidf__max_features': 10000}
CV score=0.626
evaluating SGDClassifier...


...

Best hyperparameters: {'clf__alpha': 0.0001, 'clf__max_iter': 1000, 'clf__penalty': 'l1', 'tfidf__max_df': 0.75}
CV score=0.710
evaluating Naive Bayes...


GridSearchCV(cv=10, error_score='raise',
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function my_tokenizer_function at 0x00000208A53F6310>)),
                                       ('norm', Normalizer()),
                                       ('clf', MultinomialNB())]),
             param_grid={'clf__alpha': [0.0001, 0.001, 0.01, 0.1],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters: {'clf__alpha': 0.01, 'tfidf__max_df': 0.5}
CV score=0.508
evaluating Linear SVC...


GridSearchCV(cv=10, error_score='raise',
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(tokenizer=<function my_tokenizer_function at 0x00000208A53F6310>)),
                                       ('normalize', Normalizer()),
                                       ('classifier', LinearSVC())]),
             param_grid={'classifier__C': [0.1, 1, 10],
                         'classifier__max_iter': [1000, 2000, 3000],
                         'tfidf__max_df': [0.5, 0.75, 1.0]},
             scoring='f1_macro')

Best hyperparameters: {'classifier__C': 10, 'classifier__max_iter': 1000, 'tfidf__max_df': 0.75}
CV score=0.642
The best model is: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.75,
                                 tokenizer=<function my_tokenizer_function at 0x00000208A53F6310>)),
                ('normalize', Normalizer()),
                ('clf', SGDClassifier(penalty='l1', random_state=42))]) with f1 score of: 0.710115679911672.


# Finding the best model

In [16]:
# best params - SGD Classifier
param_grid = {
    'clf__alpha': [0.0001, 0.001],
    'clf__penalty': ['l1', 'l2'],
    'clf__loss': ['hinge', 'log'],
    'clf__max_iter': [10,22,50],
    'clf__learning_rate': ['constant', 'optimal'],
    'clf__eta0': [0.1, 1, 10],
    'clf__validation_fraction': [0.1, 0.2],
    'clf__n_iter_no_change': [3, 6]
}

SGDClassifier_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer_function,max_df=0.73,ngram_range=(1,1))),
    ('normalize', preprocessing.Normalizer(norm='l2')),
    ('clf', SGDClassifier(random_state=42))
])

# perform grid search
grid_search = GridSearchCV(SGDClassifier_pipeline, param_grid, cv=10, scoring='f1_macro')
grid_search.fit(X_train, y_train)

print("Best hyperparameters are ", grid_search.best_params_)
print("Cross-validation f1 score is ", grid_search.best_score_)


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_df=0.73,
                                                        tokenizer=<function my_tokenizer_function at 0x00000208A53F6310>)),
                                       ('normalize', Normalizer()),
                                       ('clf',
                                        SGDClassifier(random_state=42))]),
             param_grid={'clf__alpha': [0.0001, 0.001],
                         'clf__eta0': [0.1, 1, 10],
                         'clf__learning_rate': ['constant', 'optimal'],
                         'clf__loss': ['hinge', 'log'],
                         'clf__max_iter': [10, 22, 50],
                         'clf__n_iter_no_change': [3, 6],
                         'clf__penalty': ['l1', 'l2'],
                         'clf__validation_fraction': [0.1, 0.2]},
             scoring='f1_macro')

Best hyperparameters are  {'clf__alpha': 0.0001, 'clf__eta0': 0.1, 'clf__learning_rate': 'optimal', 'clf__loss': 'hinge', 'clf__max_iter': 22, 'clf__n_iter_no_change': 6, 'clf__penalty': 'l1', 'clf__validation_fraction': 0.1}
Cross-validation f1 score is  0.7207252744468724


##### Average f1 Score- tfid

In [18]:
best_model_tfidf = Pipeline(
    [
        ('tfidf', TfidfVectorizer(tokenizer=my_tokenizer_function,max_df=0.73, ngram_range=(1, 1))),
        ('normalize', preprocessing.Normalizer(norm='l2')),
        ('clf', SGDClassifier(max_iter=22, random_state=42, alpha=0.0001, penalty='l1', loss='hinge', learning_rate='optimal', eta0=1, validation_fraction=0.1, n_iter_no_change=6,))
    ])

best_model_tfidf.fit(X_train, y_train)
f1_scores = cross_val_score(best_model_tfidf, X_train, y_train, cv=10, scoring='f1_macro')
avg_f1_score = np.mean(f1_scores)
print("Average f1 score is ", avg_f1_score)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.73,
                                 tokenizer=<function my_tokenizer_function at 0x00000208A53F6310>)),
                ('normalize', Normalizer()),
                ('clf',
                 SGDClassifier(eta0=1, max_iter=22, n_iter_no_change=6,
                               penalty='l1', random_state=42))])

Average f1 score is  0.7207252744468724


##### Average f1 Score - CountVectorizer

In [19]:
# trying with countvectorizer
best_model_countvectorizer = Pipeline(
    [
        ('tfidf', CountVectorizer(tokenizer=my_tokenizer_function, max_df=0.73, ngram_range=(1, 1))),
        ('normalize', preprocessing.Normalizer(norm='l2')),
        ('clf', SGDClassifier(max_iter=22, random_state=42, alpha=0.0001, penalty='l1', loss='hinge',learning_rate='optimal', eta0=1, validation_fraction=0.1, n_iter_no_change=6,))
    ])

best_model_countvectorizer.fit(X_train, y_train)
f1_scores = cross_val_score(
    best_model_countvectorizer, X_train, y_train, cv=10, scoring='f1_macro')
avg_f1_score = np.mean(f1_scores)
print("Average f1 score is ", avg_f1_score)


Pipeline(steps=[('tfidf',
                 CountVectorizer(max_df=0.73,
                                 tokenizer=<function my_tokenizer_function at 0x00000208A53F6310>)),
                ('normalize', Normalizer()),
                ('clf',
                 SGDClassifier(eta0=1, max_iter=22, n_iter_no_change=6,
                               penalty='l1', random_state=42))])

Average f1 score is  0.6866805870760653


In [23]:
# use the best estimator model for learning
y_pred = best_model_tfidf.predict(df_test["story"])
y_pred = np.where(y_pred == 0, 'm', 'f')
pd.DataFrame({'id_test': df_test["test_example_id"], 'y_pred': y_pred}).head(5)
pd.DataFrame({'id_test': df_test["test_example_id"], 'y_pred': y_pred}).tail(5)

,id_test,y_pred
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f


,id_test,y_pred
318,318,m
319,319,m
320,320,m
321,321,m
322,322,m


In [24]:
# dataframe with the test results
df_predicted = pd.DataFrame(
    {'test_example_id': df_test['test_example_id'], 'predicted_category': y_pred})
df_predicted


,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [25]:
df_predicted.to_csv('classification_results.csv',index=False)